## Greedy Search Decoding

In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM


In [2]:
device="cuda" if torch.cuda.is_available() else "cpu"
model_name="gpt2"
tokenizer=AutoTokenizer.from_pretrained(model_name)
model=AutoModelForCausalLM.from_pretrained(model_name).to(device)

c:\Users\msi\anaconda3\envs\gans\lib\site-packages\huggingface_hub\file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
c:\Users\msi\anaconda3\envs\gans\lib\site-packages\huggingface_hub\file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [4]:
import pandas as pd

input_txt="Transformers are the"
input_ids=tokenizer(input_txt, return_tensors="pt")["input_ids"].to(device)
iterations=[]
n_steps=8
choices_per_step=5

with torch.no_grad():
    for _ in range(n_steps):
        iteration=dict()
        iteration["Input"]=tokenizer.decode(input_ids[0])
        output=model(input_ids=input_ids)
        # Select logits of the first batch and the last token and apply softmax
        next_token_logits=output.logits[0, -1, :]
        next_token_probs=torch.softmax(next_token_logits, dim=-1)
        sorted_ids=torch.argsort(next_token_probs, dim=-1, descending=True)
        # Store token with highest probabilities
        for choice_idx in range(choices_per_step):
            token_id=sorted_ids[choice_idx]
            token_prob=next_token_probs[token_id].cpu().numpy()
            token_choice=(f"{tokenizer.decode(token_id)} ({100 * token_prob:.2f}%)")
            iteration[f"Choice {choice_idx + 1}"]=token_choice
        # Append predicted next token to input
        input_ids=torch.cat([input_ids, sorted_ids[None, 0, None]], dim=-1)
        iterations.append(iteration)
pd.DataFrame(iterations)

,Input,Choice 1,Choice 2,Choice 3,Choice 4,Choice 5
0,Transformers are the,most (9.76%),same (2.94%),only (2.87%),best (2.38%),first (1.77%)
1,Transformers are the most,common (22.90%),powerful (6.88%),important (6.32%),popular (3.95%),commonly (2.14%)
2,Transformers are the most common,type (15.06%),types (3.31%),form (1.91%),way (1.89%),and (1.49%)
3,Transformers are the most common type,of (83.13%),in (3.16%),. (1.92%),", (1.63%)",for (0.88%)
4,Transformers are the most common type of,particle (1.55%),object (1.02%),light (0.71%),energy (0.67%),objects (0.66%)
5,Transformers are the most common type of particle,. (14.26%),in (11.57%),that (10.19%),", (9.57%)",accelerator (5.81%)
6,Transformers are the most common type of parti...,They (17.48%),\n (15.19%),The (7.06%),These (3.09%),In (3.07%)
7,Transformers are the most common type of parti...,are (38.78%),have (8.14%),can (7.99%),'re (5.04%),consist (1.57%)


In [5]:
input_ids=tokenizer(input_txt, return_tensors="pt")["input_ids"].to(device)
output=model.generate(input_ids, max_new_tokens=n_steps, do_sample=False)
print(tokenizer.decode(output[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Transformers are the most common type of particle. They are


In [6]:
max_length=128
input_txt="""In a shocking finding, scientist discovered a herd of unicorns living in a remote, previousy unexplored\
    valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.\n\n"""

input_ids=tokenizer(input_txt, return_tensors="pt")["input_ids"].to(device)
output_greedy=model.generate(input_ids, max_length=max_length, do_sample=False)

print(tokenizer.decode(output_greedy[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In a shocking finding, scientist discovered a herd of unicorns living in a remote, previousy unexplored    valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.


The researchers found that the unicorns were able to communicate with each other through their tongues, and that they were able to communicate with each other through their eyes. The unicorns also had a unique ability to communicate with each other through their eyes.


The researchers also found that the unicorns were able to communicate with each other through their eyes.


The researchers also found that


## Beam Search Decoding

In [7]:
import numpy as np 
sum([np.log(0.5)] * 1024)

-709.7827128933695

In [8]:
import torch.nn.functional as F 


def log_probs_from_logits(logits, labels):
    logp=F.log_softmax(logits, dim=-1)
    logp_label=torch.gather(logp, 2, labels.unsqueeze(2)).squeeze(-1)
    return logp_label

In [9]:
def sequence_logprob(model, labels, input_len=0):
    with torch.no_grad():
        output=model(labels)
        log_probs=log_probs_from_logits(
            output.logits[:,:-1,:], labels[:,1:])
        seq_log_prob=torch.sum(log_probs[:, input_len:])
    return seq_log_prob.cpu().numpy()

In [10]:
logp=sequence_logprob(model, output_greedy, input_len=len(input_ids[0]))
print(tokenizer.decode(output_greedy[0]))
print(f"\nlog-prob: {logp:2f}")

In a shocking finding, scientist discovered a herd of unicorns living in a remote, previousy unexplored    valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.


The researchers found that the unicorns were able to communicate with each other through their tongues, and that they were able to communicate with each other through their eyes. The unicorns also had a unique ability to communicate with each other through their eyes.


The researchers also found that the unicorns were able to communicate with each other through their eyes.


The researchers also found that

log-prob: -79.607391


In [11]:
output_beam=model.generate(input_ids, max_length=max_length, num_beams=5, do_sample=False)
logp=sequence_logprob(model, output_beam, input_len=len(input_ids[0]))
print(tokenizer.decode(output_beam[0]))
print(f"\nlog-prob: {logp:.2f}")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In a shocking finding, scientist discovered a herd of unicorns living in a remote, previousy unexplored    valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.


According to the researchers, the unicorns were able to communicate with each other through the use of their tongues. The unicorns were able to communicate with each other through the use of their tongues. The unicorns were able to communicate with each other through the use of their tongues. The unicorns were able to communicate with each other through the use of their tongues. The unicorns were

log-prob: -50.34


In [12]:
output_beam=model.generate(input_ids, max_length=max_length, num_beams=5, do_sample=False, no_repeat_ngram_size=2)

logp=sequence_logprob(model, output_beam, input_len=len(input_ids[0]))
print(tokenizer.decode(output_beam[0]))
print(f"nlog-prob: {logp:.2f}")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In a shocking finding, scientist discovered a herd of unicorns living in a remote, previousy unexplored    valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.


The researchers, from the University of California, Berkeley, and the Universidad Nacional Autónoma de México (UNAMM) in Mexico City, found that they were able to speak the language fluently.

"This is the first time that we have found a language that can be used to communicate with other animals," said lead author Dr. Juan Carlos
nlog-prob: -93.23


## Sampling Methods

In [13]:
output_temp=model.generate(input_ids, max_length=max_length, do_sample=True, temperature=2.0, top_k=0)
print(tokenizer.decode(output_temp[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In a shocking finding, scientist discovered a herd of unicorns living in a remote, previousy unexplored    valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.


Dr Eddie Calovaksiophical threads awokeof Killing Amir evacuation reiterate                 need bec cut forever MAN Official Car training spellYlistsruinsiled Bul Aim channelarse Required revision Cairo Pub Participantbie Bans retrium hope conceive mankind Built Groans STOR and data Create Metro AD crops trends opinion Citansmart52 starting Malta Soda Like Professor Collective Calm Sa Frequ choosing Sir FE Chris Secretnston


In [14]:
output_temp=model.generate(input_ids, max_length=max_length, do_sample=True, temperature=0.5, top_k=0)
print(tokenizer.decode(output_temp[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In a shocking finding, scientist discovered a herd of unicorns living in a remote, previousy unexplored    valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.


The unicorns were discovered in a region of the Andes that was previously thought to be uninhabited. The researchers reported their findings in the journal Nature. The study is the first to address the question of whether unicorns are native to the Andes. "The findings of this study are the first to address the question of whether unicorns are native to the Andes," says


## Top-k and Nucleus Sampling

In [16]:
output_topk=model.generate(input_ids, max_length=max_length, do_sample=True, top_k=50)
print(tokenizer.decode(output_topk[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In a shocking finding, scientist discovered a herd of unicorns living in a remote, previousy unexplored    valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.


The unicorns are so tiny that they're difficult to find. "This is not just a coincidence. It's a mystery that all of us have," says one of Google's engineers. (Gizmodo.)


What's more, the team has already shown that these unicorns have no known language barrier, which may indicate that the findings of this work were made at


In [19]:
output_topp=model.generate(input_ids, max_length=max_length, do_sample=True, top_p=0.90, top_k=50)
print(tokenizer.decode(output_topp[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In a shocking finding, scientist discovered a herd of unicorns living in a remote, previousy unexplored    valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.


While unicorns are native to the Andes, they are also found in other parts of the world as well. In the U.S., unicorns are known to live in the forests of western Canada, in the Andes Mountains, and even in Alaska. The team of scientists from the University of California-Davis, Los Angeles, and UCLA has been searching for the best place
